# Load Data

In [1]:
from pyearth import Earth
from DLtools.Data import load_data,instant_data,check_specific_col,station_sel

import numpy as np
import pandas as pd

In [2]:
# loaddata = load_data()
# df_d = loaddata.daily()
# df_h = loaddata.hourly()
###################################
loading = instant_data()
df,mode = loading.hourly_instant(),'hour'
# df,mode = loading.daily_instant(),'day'


In [3]:
st = 'CPY012'
target,start_p,stop_p,host_path=station_sel(st,mode)
if mode =='hour': n_past,n_future = 24*7,72
elif mode =='day': n_past,n_future = 60,30

data = df[start_p:stop_p]
n_out=n_future

In [4]:
data['Day'] = data.index.dayofyear #add day
data = data.interpolate(limit=300000000,limit_direction='both').astype('float32')#interpolate neighbor first, for rest NA fill with mean() #.apply(lambda x: x.fillna(x.mean()),axis=0)

# Missing data handle

In [5]:
# # data = df_h['2013-01-01':'2017-12-31']

# # limit 80% avaliable data only
# for col in data.columns:
#     if data[col].count()<len(data[col])*.8:
#         data = data.drop(col, axis=1)
# # Fill remain with mean
# data = data.apply(lambda x: x.fillna(x.mean()),axis=0)
# data.head()

In [6]:
def to_supervise(data,target,n_out):
    data[target]=data[target].shift(-n_out)
    data = data.astype('float64').dropna()
    X = data.drop([target],axis=1)
    xlabels = list(X.columns)
    X = X.values
    y = data[target].values
    return X,y,xlabels

In [7]:
X,y,xlabels = to_supervise(data,target,n_out)

# MARS feature selection (from all stations)

In [8]:
# #scale data since it's cannot run MARS in general
# from sklearn.preprocessing import MinMaxScaler
# # SCALE
# scaler = MinMaxScaler()
# data[data.columns] = scaler.fit_transform(data[data.columns])


In [9]:
#Fit an Earth model
criteria = ('rss', 'gcv', 'nb_subsets')
model = Earth(enable_pruning = True,
            #   max_degree=3,
            #  max_terms=20,
              minspan_alpha=.5,
              feature_importance_type=criteria,
              verbose=True)
model.fit(X,y,xlabels=xlabels)


Beginning forward pass
---------------------------------------------------------------
iter  parent  var  knot  mse       terms  gcv    rsq    grsq   
---------------------------------------------------------------
0     -       -    -     0.572393  1      0.572  0.000  0.000  
1     0       23   28712  0.089260  3      0.089  0.844  0.844  
2     0       51   23859  0.081194  5      0.081  0.858  0.858  
3     0       116  22712  0.076470  7      0.077  0.866  0.866  
4     0       41   10493  0.072758  9      0.073  0.873  0.873  
5     0       25   13859  0.069320  11     0.069  0.879  0.879  
6     0       177  17264  0.067387  13     0.068  0.882  0.882  
7     0       138  27060  0.065968  15     0.066  0.885  0.885  
8     0       69   19658  0.064536  17     0.065  0.887  0.887  
9     0       111  21075  0.063560  19     0.064  0.889  0.889  
10    0       112  18257  0.062494  21     0.063  0.891  0.891  
11    0       65   30102  0.061628  23     0.062  0.892  0.892  
12    

Earth(feature_importance_type=('rss', 'gcv', 'nb_subsets'), minspan_alpha=0.5,
      verbose=True)

In [10]:
# #Print the model
# print(model.trace())
# print(model.summary())

* Feature Importance

In [11]:
print(model.summary_feature_importances(sort_by='rss')[:500])

                  rss    gcv    nb_subsets
CPY009_wl         0.92   0.92   0.10          
CPY008_wl         0.03   0.03   0.05          
NAN008_wl         0.01   0.01   0.05          
CPY011_wl         0.01   0.01   0.02          
DNP019_temp       0.00   0.00   0.05          
DNP025_temp       0.00   0.00   0.05          
GLF001_wl         0.00   0.00   0.05          
DNP004_press      0.00   0.00   0.05          
DIV006_wl         0.00   0.00   0.05          
DNP010_humid      0.00   0.00   0.


In [12]:
def toDF(rank):
    name,rss,gcv,nb_sub= list(),list(),list(),list()
    for i in range(len(rank)):
        if i%4==0:
            name.append(rank[i])
        elif i%4==1:
            rss.append(rank[i])
        elif i%4==2:
            gcv.append(rank[i])
        elif i%4==3:
            nb_sub.append(rank[i])
    data = {'feature':name,
    'rss':rss,
    'gcv':gcv,
    'nb_subset':nb_sub}
    score = pd.DataFrame(data)
    return score

In [13]:
nbsub = model.summary_feature_importances(sort_by='nb_subsets')[:2000].split()[3:83]
gcv = model.summary_feature_importances(sort_by='gcv')[:2000].split()[3:83]
rss = model.summary_feature_importances(sort_by='rss')[:2000].split()[3:83]


In [14]:
rss = toDF(rss)
gcv = toDF(gcv)
nbsub = toDF(nbsub)

In [15]:
#From top 20 rss,gcv,nbsub
top20 = pd.concat([rss,gcv,nbsub],ignore_index=True).drop_duplicates('feature')
top20

,feature,rss,gcv,nb_subset
0,CPY009_wl,0.92,0.92,0.10
1,CPY008_wl,0.03,0.03,0.05
2,NAN008_wl,0.01,0.01,0.05
3,CPY011_wl,0.01,0.01,0.02
4,DNP019_temp,0.00,0.00,0.05
5,DNP025_temp,0.00,0.00,0.05
6,GLF001_wl,0.00,0.00,0.05
7,DNP004_press,0.00,0.00,0.05
8,DIV006_wl,0.00,0.00,0.05
9,DNP010_humid,0.00,0.00,0.05


In [16]:
# top20.to_csv('/home/song/Public/Song/Work/Thesis/MAR/[CPY012]_featurelist_MAR_daily_7d.csv',index=False)

# LOOP

In [17]:
conclude_df=pd.DataFrame()
for n_out in range(1,n_out):
    X,y,xlabels = to_supervise(data,target,n_out)
    criteria = ('rss', 'gcv', 'nb_subsets')
    model = Earth(enable_pruning = True,
                #   max_degree=3,
                #  max_terms=20,
                minspan_alpha=.5,
                feature_importance_type=criteria,
                verbose=True)
    model.fit(X,y,xlabels=xlabels)
    nbsub = model.summary_feature_importances(sort_by='nb_subsets')[:2000].split()[3:83]
    gcv = model.summary_feature_importances(sort_by='gcv')[:2000].split()[3:83]
    rss = model.summary_feature_importances(sort_by='rss')[:2000].split()[3:83]
    
    rss,gcv,nbsub = toDF(rss),toDF(gcv),toDF(nbsub)
    top20=pd.concat([rss,gcv,nbsub],ignore_index=True)
    top20 = pd.concat([rss,gcv,nbsub],ignore_index=True).drop_duplicates('feature')
    top20['timestep'] = n_out

    #ADDED combine all result
    conclude_df = pd.concat([conclude_df,top20],ignore_index=True)
    if mode=='day':
        conclude_df.drop_duplicates('feature').to_csv('/home/song/Public/Song/Work/Thesis/MAR/[CPY012]featurelist_MAR_daily.csv',index=False)
        conclude_df.to_csv('/home/song/Public/Song/Work/Thesis/MAR/[CPY012]featurelist_MAR_daily_Fullreport.csv',index=False)

    elif mode=='hour':
        conclude_df.drop_duplicates('feature').to_csv('/home/song/Public/Song/Work/Thesis/MAR/[CPY012]featurelist_MAR_hourly.csv',index=False)
        conclude_df.to_csv('/home/song/Public/Song/Work/Thesis/MAR/[CPY012]featurelist_MAR_hourly_Fullreport.csv',index=False)

05  0.824  0.824  
50    44  23     0.11  0.107  0.821  0.820  
51    37  22     0.11  0.110  0.818  0.817  
52    40  21     0.11  0.112  0.814  0.814  
53    31  20     0.11  0.114  0.810  0.810  
54    24  19     0.12  0.116  0.806  0.805  
55    45  18     0.12  0.118  0.803  0.802  
56    61  17     0.12  0.121  0.799  0.798  
57    58  16     0.12  0.122  0.797  0.796  
58    16  15     0.13  0.126  0.790  0.790  
59    71  14     0.13  0.127  0.788  0.788  
60    15  13     0.13  0.131  0.782  0.782  
61    30  12     0.13  0.134  0.777  0.777  
62    28  11     0.14  0.137  0.771  0.771  
63    54  10     0.14  0.143  0.762  0.761  
64    59  9      0.15  0.155  0.741  0.741  
65    17  8      0.16  0.156  0.740  0.740  
66    6   7      0.17  0.175  0.708  0.708  
67    10  6      0.18  0.177  0.704  0.704  
68    52  5      0.20  0.203  0.661  0.661  
69    50  4      0.27  0.272  0.546  0.546  
70    48  3      0.30  0.303  0.493  0.493  
71    1   2      0.59  0.588  0.018 